In [87]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np
from hydra import initialize, compose

from assembly_tools import load_experimental_grid, format_table, extract_raw_performance

In [88]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Assembly of experimental results 2


In [91]:
# load default params. They can be changed however you want.
with initialize(version_base=None, config_path="config"):
    cfg = compose(config_name='extract.yaml')
    print(cfg)
general_params =  ["resolution_value", "ds_name", "method_hps.max_lag"]
hp_stack = []
performance_stack = []
scoring = 'Max individual F1'
cfg.ds_order = ["random_5"]

{'metrics': ['Max Acc thresh', 'Max Acc', 'Max individual Acc thresh', 'Max individual Acc', 'Null Acc', 'Max F1 thresh', 'Max F1', 'Max individual F1 thresh', 'Max individual F1', 'Null F1', 'AUROC', 'Individual AUROC', 'Null AUROC', 'runtime'], 'hp_list': ['data_preprocess.resolution', 'data_preprocess.normalize', 'label_path', 'method.max_lag', 'method.var_absolute_values', 'method.filter_mode', 'method.architecture'], 'rename': {'resolution_value': 'data_preprocess.resolution', 'method_hps.var_absolute_values': 'method.var_absolute_values', 'method_hps.filter_mode': 'method.filter_mode', 'method_hps.max_lag': 'method.max_lag', 'method_hps.cp_architecture': 'method.architecture', 'ds_name': 'label_path', 'normalize': 'data_preprocess.normalize', 'window_data_month_value': 'data_preprocess.subset_month', 'window_data_year_value': 'data_preprocess.subset_year'}, 'ds_order': ['close_3', 'close_5', 'root_cause_3', 'root_cause_5', '1_random_3', '1_random_5', 'confounder_3', 'confounder_5

In [92]:
for name in ["reverse", "cross", "combo", "var_", "varlingam","pcmci","dyno","cp"]:
    data = pd.read_csv("grid_export_exp2/" + name + ".csv", index_col=0)
    res = ["data_path",  "data_preprocess.subset_year"]
    formatted = format_table(data,cfg, scoring= scoring, restriction=res)
    lines = extract_raw_performance(formatted,cfg,scoring,name="", restriction=res)
    lines.columns = [name]
    hp_stack.append(formatted)
    performance_stack.append(lines)

Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')


In [94]:
# We ran this completely distributed so we load the raw files here.
data = load_experimental_grid(mypath=cfg.data_path + "exp2/", method_name="cdmi").T

In [95]:
out = pd.concat(performance_stack,axis=1).T

In [96]:
out

,_resources/rivers_ts_east_germany.csv-2021,_resources/rivers_ts_east_germany.csv-False,_resources/rivers_flood.csv-False
reverse,0.636681,0.636681,0.567248
cross,0.512241,0.610808,0.593368
combo,0.624228,0.622012,0.596208
var_,0.704915,0.684465,0.676528
varlingam,0.58635,0.613842,0.619771
pcmci,0.580214,0.498137,0.533264
dyno,0.494512,0.691835,0.707216
cp,0.530792,0.548164,0.60005


In [97]:
out = out[[out.columns[1], out.columns[0], out.columns[2]]]
out.columns = ["Full TS", "No Rain", "Flood"]
out.loc["cdmi"] = [data["Max individual F1"].mean(), np.nan, np.nan]
out.index = ["RP", "CC", "RPCC", "VAR", "Varlingam", "PCMCI", "Dynotears", "CP", "CDMI"]
out = out.astype(float).round(3)

/tmp/ipykernel_2350123/1754352668.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out.loc["cdmi"] = [data["Max individual F1"].mean(), np.nan, np.nan]


In [98]:
# switch order
out =out.loc[list(out.index[:-2]) + ["CDMI"] + ["CP"]]

In [99]:
style = out.style.highlight_max(axis=1,  props='color:black;font-weight:bold')
style = style.format("{:.2f}")
latex_str = style.to_latex(convert_css=True)
display(HTML(style.to_html()))
print(latex_str.replace("nan", "-*"))

,Full TS,No Rain,Flood
RP,0.64,0.64,0.57
CC,0.61,0.51,0.59
RPCC,0.62,0.62,0.60
VAR,0.68,0.70,0.68
Varlingam,0.61,0.59,0.62
PCMCI,0.50,0.58,0.53
Dynotears,0.69,0.49,0.71
CDMI,0.69,nan,nan
CP,0.55,0.53,0.60


\begin{tabular}{lrrr}
 & Full TS & No Rain & Flood \\
RP & \color{black} \bfseries 0.64 & \color{black} \bfseries 0.64 & 0.57 \\
CC & \color{black} \bfseries 0.61 & 0.51 & 0.59 \\
RPCC & 0.62 & \color{black} \bfseries 0.62 & 0.60 \\
VAR & 0.68 & \color{black} \bfseries 0.70 & 0.68 \\
Varlingam & 0.61 & 0.59 & \color{black} \bfseries 0.62 \\
PCMCI & 0.50 & \color{black} \bfseries 0.58 & 0.53 \\
Dynotears & 0.69 & 0.49 & \color{black} \bfseries 0.71 \\
CDMI & \color{black} \bfseries 0.69 & -* & -* \\
CP & 0.55 & 0.53 & \color{black} \bfseries 0.60 \\
\end{tabular}

